In [8]:
import torch
import torch.nn.functional as F

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('medicine_dataset.csv')
df.head(10)

C:\Users\deepa\AppData\Local\Temp\ipykernel_12728\3516404252.py:1: DtypeWarning: Columns (42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('medicine_dataset.csv')


,id,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,sideEffect2,...,sideEffect41,use0,use1,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class
0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN
1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides
2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,Diarrhea,...,NaN,Treatment of Cough with mucus,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN
3,4,allegra 120mg tablet,Lcfex Tablet,Etofex 120mg Tablet,Nexofex 120mg Tablet,Fexise 120mg Tablet,Histafree 120 Tablet,Headache,Drowsiness,Dizziness,...,NaN,Treatment of Sneezing and runny nose due to allergies,Treatment of Allergic conditions,NaN,NaN,NaN,Diphenylmethane Derivative,No,RESPIRATORY,H1 Antihistaminics (second Generation)
4,5,avil 25 tablet,Eralet 25mg Tablet,NaN,NaN,NaN,NaN,Sleepiness,Dryness in mouth,NaN,...,NaN,Treatment of Allergic conditions,NaN,NaN,NaN,NaN,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation)
5,6,allegra-m tablet,Emlukast-FX Tablet,LCFEX-Mont Tablet,Fixar 10mg/120mg Tablet,Histakind-M Tablet,Histafree-M Tablet,Nausea,Diarrhea,Vomiting,...,NaN,Treatment of Sneezing and runny nose due to allergies,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN
6,7,amoxyclav 625 tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,Diarrhea,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,NaN,No,ANTI INFECTIVES,NaN
7,8,azee 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,Abdominal pain,...,NaN,Treatment of Bacterial infections,NaN,NaN,NaN,NaN,Macrolides,No,ANTI INFECTIVES,Macrolides
8,9,atarax 25mg tablet,HD Zine 25mg Tablet,Hyzox 25 Tablet,Hizet 25mg Tablet,Hydil 25mg Tablet,Zyzine 25mg Tablet,Sedation,Nausea,Vomiting,...,NaN,Treatment of Anxiety,Treatment of Skin conditions with inflammation & itching,NaN,NaN,NaN,Piperazine Derivative,No,RESPIRATORY,H1 Antihistaminics (First Generation)
9,10,ascoril d plus syrup sugar free,Arnikof D Syrup,Cofsolve-D Syrup,Tucin D Syrup,Akof-D Syrup Sugar Free,Krisbro D Syrup,Nausea,Vomiting,Loss of appetite,...,NaN,Treatment of Dry cough,NaN,NaN,NaN,NaN,NaN,No,RESPIRATORY,NaN


In [11]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248218 entries, 0 to 248217
Data columns (total 58 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 248218 non-null  int64 
 1   name               248218 non-null  object
 2   substitute0        238621 non-null  object
 3   substitute1        233867 non-null  object
 4   substitute2        230233 non-null  object
 5   substitute3        226856 non-null  object
 6   substitute4        223962 non-null  object
 7   sideEffect0        248218 non-null  object
 8   sideEffect1        238416 non-null  object
 9   sideEffect2        229500 non-null  object
 10  sideEffect3        207638 non-null  object
 11  sideEffect4        163560 non-null  object
 12  sideEffect5        131258 non-null  object
 13  sideEffect6        91857 non-null   object
 14  sideEffect7        67750 non-null   object
 15  sideEffect8        48506 non-null   object
 16  sideEffect9        3

In [14]:
selected_columns = ['name', 'sideEffect1', 'use0']
selected_columns = [col for col in selected_columns if col in df.columns]
df_selected = df[selected_columns].fillna("Not Available")
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 200)
print(df_selected.head(10))

                              name       sideEffect1                                                   use0
0         augmentin 625 duo tablet            Nausea                      Treatment of Bacterial infections
1              azithral 500 tablet            Nausea                      Treatment of Bacterial infections
2                 ascoril ls syrup          Vomiting                          Treatment of Cough with mucus
3             allegra 120mg tablet        Drowsiness  Treatment of Sneezing and runny nose due to allergies
4                   avil 25 tablet  Dryness in mouth                       Treatment of Allergic conditions
5                 allegra-m tablet          Diarrhea  Treatment of Sneezing and runny nose due to allergies
6             amoxyclav 625 tablet            Nausea                      Treatment of Bacterial infections
7                  azee 500 tablet            Nausea                      Treatment of Bacterial infections
8               atarax 25mg 

In [15]:
df_selected((df[sideEffect1=='vomiting']))

NameError: name 'sideEffect1' is not defined

In [ ]:
df_cleaned1 = df_selected.drop_duplicates(subset='name', keep='first')
df_cleaned = df_cleaned1.drop(columns=['sideEffect1'])
print(df_cleaned)

In [ ]:
import torch
import os
import joblib
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification


label_encoder_path = r"C:\Users\deepa\Downloads\label_encoder_prediction.pkl"
if os.path.exists(label_encoder_path):
    label_encoder = joblib.load(label_encoder_path)
    print("LabelEncoder loaded successfully.")
    num_labels = len(label_encoder.classes_)  
else:
    print(f"Error: File not found at {label_encoder_path}")
    num_labels = 530  


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  


file_path = r"C:\Users\deepa\Downloads\bert_finetuned_model_prediction.pkl"
if os.path.exists(file_path):
    state_dict = torch.load(file_path, map_location=device)
    num_labels = state_dict['classifier.bias'].shape[0]  

    
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    print(f"Model loaded successfully with {num_labels} labels.")
else:
    print(f"Error: File not found at {file_path}")



In [ ]:
def predict_use(medicine_name):
    model.eval()
    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            medicine_name,
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(F.softmax(outputs.logits, dim=1), dim=1)
        return label_encoder.inverse_transform(prediction.cpu().numpy())[0]



In [ ]:
medicine_name = input("Enter Medicine Name: ")
print(f"Predicted use: {predict_use(medicine_name)}")
